# Сравнение запросов для работы с GPT от Yandex и OpenAI

In [4]:
!pip install python-dotenv

Импортируем настройки для работы с yandex и openai

In [1]:
from pydantic_settings import BaseSettings
import logging
from dotenv import load_dotenv
import os

load_dotenv(dotenv_path='../.env')

log_levels = {
    'DEBUG': logging.DEBUG,
    'INFO': logging.INFO,
    'WARNING': logging.WARNING,
    'ERROR': logging.ERROR
}



class Settings(BaseSettings):
    api_name: str = "NLP API"
    logging_level: str = "INFO"

    sentry_dsn: str = ...
    sentry_enable_tracing: bool = True

    elastic_host: str = ...
    elastic_port: int = 9200
    
    project_name: str = ...
    sqlite_path: str = ...
    postgres_version: str = ...
    postgres_ports: str = ...
    postgres_port: str = ...
    postgres_host: str = ...
    postgres_user: str = ...
    postgres_password: str = ...
    postgres_schema_2: str = ...
    postgres_db: str = ...
    postgres_name: str = ...
    elastic_movies_index: str = ...
    stack_version: str = ...
    elastic_cluster_name: str = ...
    memory_lock: str = ...
    elastic_ports: str = ...
    redis_ports: str = ...
    redis_host: str = ...
    redis_port: str = ...
    logstash_ports1: str = ...
    logstash_ports2: str = ...
    logstash_ports3: str = ...
    kibana_ports: str = ...
    es_public_host: str = ...
    logstash_public_host: str = ...
    logstash_host: str = ...
    logstash_port: str = ...
    kibana_public_host: str = ...
    kibana_public_url: str = ...
    elasticsearch_host: str = ...
    es_url: str = ...
    
    rabbit_mq_version: str = ...
    rabbit_mq_ports: str = ...
    rabbit_mq_amqp_ports: str = ...
    rabbit_mq_amqp_port: str = ...
    rabbit_mq_port: str = ...
    rabbit_mq_host: str = ...
    rabbit_mq_user: str = ...
    rabbit_mq_password: str = ...
    rabbit_mq_subject_queue: str = ...
    rabbit_mq_actor_queue: str = ...
    rabbit_mq_director_queue: str = ...
    rabbit_mq_rating_order_queue: str = ...
    rabbit_mq_genre_filter_queue: str = ...
    rabbit_mq_date_filter_queue: str = ...
    rabbit_mq_title_text_queue: str = ...
    rabbit_mq_description_text_queue: str = ...
    rabbit_mq_exchange: str = ...
    
    yc_api_key: str = ...
    ya_gpt_model: str = ...
    reka_api_key: str = ...
    claude_api_key: str = ...

    def get_logging_level(self) -> int:
        return log_levels.get(self.logging_level, logging.INFO)

    class Config:
        env_file = '../.env'


settings = Settings()

In [2]:
from langchain import LLMChain, PromptTemplate
from langchain_community.vectorstores import ElasticVectorSearch
from langchain_community.llms import YandexGPT
import time

# YANDEX GPT 3

## Пробуем запросить готовый запрос для Elasticsearch на английском языке

In [3]:
template = """
You are an assistant that converts natural language queries into Elasticsearch queries.
Convert the following natural language query to an Elasticsearch query:

Query: {query}

Please translate any lanquage to english if needed. Write please only the Elasticsearch query.

There are 3 indices: movies, genres and persons. Persons could be directors or actors.
"""

prompt = PromptTemplate(template=template, input_variables=["query"])
llm = YandexGPT(api_key=settings.yc_api_key,
                model_uri="gpt://b1gsfbseql2uigg6q996/yandexgpt/latest")
llm_chain = prompt | llm
current_time = time.time()
query = "Найти фильмы с участием актера Джонни Деппа"

elasticsearch_query = llm_chain.invoke(query)
print (f'Время выполнения: {time.time() - current_time} секунд')
print('='*10)
print(elasticsearch_query)


Время выполнения: 2.6231157779693604 секунд
```
{
  "query": {
    "match": {
      "persons.name": "Джонни Депп"
    }
  },
  "size": 20,
  "from": 0,
  "_source": ["title", "release_year"]
}
```


Время выполнения более 4,5 секунд. Нас не устраивает такой результат.

Также не переводятся запросы

## Пробуем запросить готовый запрос для Elasticsearch на руссом языке

In [4]:
template = """
Вы помощник, который преобразует запросы на естественном языке в запросы Elasticsearch.
Преобразуйте следующий запрос на естественном языке в запрос Elasticsearch:

Запрос: {query}

Пожалуйста, переведите любой язык на английский, если это необходимо. Напишите, пожалуйста, только запрос Elasticsearch.

Существует 3 индекса: movies (фильмы), genres (жанры) и persons (люди). Люди могут быть режиссерами или актерами.
"""

prompt = PromptTemplate(template=template, input_variables=["query"])
llm = YandexGPT(api_key=settings.yc_api_key,
                model_uri="gpt://b1gsfbseql2uigg6q996/yandexgpt/latest")
llm_chain = prompt | llm
current_time = time.time()
query = "Найти фильмы с участием актера Джонни Деппа"

elasticsearch_query = llm_chain.invoke(query)
print (f'Время выполнения: {time.time() - current_time} секунд')
print('='*10)
print(elasticsearch_query)


Время выполнения: 5.577401876449585 секунд
Вот запрос Elasticsearch для поиска фильмов с участием актёра Джонни Деппа:

```elasticsearch
{
  "query": {
    "term": {
      "persons.name": "Johnny Depp"
    }
  },
  "size": 10,
  "from": 0,
  "sort": [
    "_score"
  ],
  "aggs": {},
  "filter": {}
}
```

Этот запрос будет искать документы в индексе `movies`, которые содержат информацию об актёре с именем Johnny Depp (Джонни Депп) в поле `persons.name`. Размер возвращаемого результата устанавливается равным 10 документам.


Время выполнения запроса на руссом языке более 4,5 секунд. Нас не устраивает такой результат.
Хотя воспиятие запроса лучше

In [17]:
## Вынесем объявление LLM в отдельную ячейку

In [5]:
llm = YandexGPT(api_key=settings.yc_api_key,
                model_uri="gpt://b1gsfbseql2uigg6q996/yandexgpt/latest",
                top_p=0.1,
                temperature=0.1,
                max_tokens=20)

In [6]:
template = """
Определи с поиском чего связан следущий запрос: поиск фильмов, поиск жанров или поиск персон.
Запрос: {query}
Если ты сомневаешься — отвечай none.
У тебя есть 4 варианта ответа: movies, genres, persons, none. Ответь, пожалуйста, одним словом.
"""
prompt = PromptTemplate(template=template, input_variables=["query"])
llm_chain = prompt | llm
current_time = time.time()
queries = ["Найти фильмы с участием актера Джонни Деппа",
           "Найти фильмы без титров",
           "Найти улиточек с именем Наталии Деппа",
           "Что-нибудь про войну"]

for query in queries:
    current_time = time.time()
    search_type = llm_chain.invoke(query)
    print(f'Время выполнения: {time.time() - current_time} секунд')
    print(search_type)
    print('=' * 10)




Время выполнения: 1.2684009075164795 секунд
movies
Время выполнения: 1.6781799793243408 секунд
movies
Время выполнения: 1.157608985900879 секунд
persons
Время выполнения: 1.0520851612091064 секунд
movies


Скорость ответа уже преемлемая


In [37]:
template = """
Определи содержится ли в данном запросе уточнение по свойству: имя человека
Запрос: {query}
Если ты сомневаешься — отвечай none.
Если есть — ответь через запятую строками для фильтра переведенными на английский язык.
Без лишних слов, имена людей через запятую или none. Например Joe Lucas, Natalie Portman.
"""
prompt = PromptTemplate(template=template, input_variables=["query"])
llm_chain = prompt | llm
current_time = time.time()
queries = ["Найти фильмы с участием актера Джонни Деппа",
           "Найти фильмы без титров",
           "Найти улиточек с именем Наталии Деппа",
           "Что-нибудь про войну"]

for query in queries:
    current_time = time.time()
    search_type = llm_chain.invoke(query)
    print(f'Время выполнения: {time.time() - current_time} секунд')
    print(search_type)
    print('=' * 10)




In [36]:
template = """
Ты компьютер, который выдаёт в командную строку список актёров через зяпятую из запроса.
Определи содержится ли в данном запросе уточнение по имени актёра.
Запрос: {query}
Если ты сомневаешься — отвечай none.
Если есть — ответь через запятую строками для фильтра переведенными на английский язык.
Без лишних слов, имена актёров через запятую или none. Например: Nikolas Cage, Natalie Portman.
"""
prompt = PromptTemplate(template=template, input_variables=["query"])
llm_chain = prompt | llm
current_time = time.time()
queries = ["Найти фильмы джоржа лукаса",
           "Найти фильмы без титров",
           "Найти улиточек с именем Наталии Деппа",
           "Что-нибудь про войну"]

for query in queries:
    current_time = time.time()
    search_type = llm_chain.invoke(query)
    print(f'Время выполнения: {time.time() - current_time} секунд')
    print(search_type)
    print('=' * 10)



In [37]:
template = """
Ты компьютер, который выдаёт в командную строку список режиссеров через зяпятую из запроса.
Определи содержится ли в данном запросе уточнение по имени режиссера.
Запрос: {query}
Если ты сомневаешься — отвечай none.
Если есть — ответь через запятую строками для фильтра переведенными на английский язык.
Без лишних слов, имена актёров через запятую или none. Например: Quentin Tarantino, Nikolas Cage.
"""
prompt = PromptTemplate(template=template, input_variables=["query"])
llm_chain = prompt | llm
current_time = time.time()
queries = ["Найти фильмы джоржа лукаса",
           "Найти фильмы без титров",
           "Найти улиточек с именем Наталии Деппа",
           "Что-нибудь про войну"]

for query in queries:
    current_time = time.time()
    search_type = llm_chain.invoke(query)
    print(f'Время выполнения: {time.time() - current_time} секунд')
    print(search_type)
    print('=' * 10)



In [40]:
template = """
Определи содержится ли в данном запросе уточнение по упорядочиванию результатов по рейтингу
Запрос: {query}
Если ты сомневаешься — отвечай none.
Отвечай отднословно: asc, desc или none
"""
prompt = PromptTemplate(template=template, input_variables=["query"])
llm_chain = prompt | llm
current_time = time.time()
queries = ["Найти фильмы с участием актера Джонни Деппа",
           "Найти фильмы без титров",
           "Найти улиточек с именем Наталии Деппа",
           "Что-нибудь про войну c высоким рейтингом"]

for query in queries:
    current_time = time.time()
    search_type = llm_chain.invoke(query)
    print(f'Время выполнения: {time.time() - current_time} секунд')
    print(search_type)
    print('=' * 10)




In [41]:
template = """
Определи содержится ли в данном запросе уточнение по дате выхода фильма
Запрос: {query}
Если ты сомневаешься — отвечай none.
Отвечай в формате: gte:2020-01-01, lte:2021-01-01 или none
"""
prompt = PromptTemplate(template=template, input_variables=["query"])
llm_chain = prompt | llm
current_time = time.time()
queries = ["Найти фильмы с участием актера Джонни Деппа",
           "Найти фильмы без титров",
           "Найти улиточек с именем Наталии Деппа после 2020 года",
           "Что-нибудь про войну c высоким рейтингом"]

for query in queries:
    current_time = time.time()
    search_type = llm_chain.invoke(query)
    print(f'Время выполнения: {time.time() - current_time} секунд')
    print(search_type)
    print('=' * 10)




In [38]:
template = """
Определи содержится ли в данном запросе уточнение по жанру или жанрам фильма
Запрос: {query}
Если ты сомневаешься — отвечай none.
допустимые значения: Action, Adventure, Fantasy, Sci-Fi, Drama, Music, Romance, Thriller, Mystery, Comedy, Animation, Family, Biography, Musical, Crime, Short, Western, Documentary, History, War, Game-Show, Reality-TV, Horror, Sport, Talk-Show, News
Отвечай в формате: значений через запятую или none
"""
prompt = PromptTemplate(template=template, input_variables=["query"])
llm_chain = prompt | llm
current_time = time.time()
queries = ["Найти фильмы с участием актера Джонни Деппа",
           "Найти фильмы без титров",
           "Найти улиточек с именем Наталии Деппа после 2020 года",
           "Что-нибудь про войну c высоким рейтингом"]

for query in queries:
    current_time = time.time()
    search_type = llm_chain.invoke(query)
    print(f'Время выполнения: {time.time() - current_time} секунд')
    print(search_type)
    print('=' * 10)




In [14]:
template = """
Определи содержатся ли в запросе слова, которые могут быть частью названия фильма

Если такие есть — перечисли их через запятую на английском. Если нет или сомневаешься — none
Отвечай в формате слова через запятую или none. Без окружающего текста и пояснений. Например: Star Wars, The Matrix.

Запрос: {query}
"""
prompt = PromptTemplate(template=template, input_variables=["query"])
llm_chain = prompt | llm
current_time = time.time()
queries = ["Найти фильмы про звездные войны",
           "Фильм спиздили",
           "Найти мультфильмы про звездные войны",
           "Что-нибудь про войну c высоким рейтингом"]

for query in queries:
    current_time = time.time()
    search_type = llm_chain.invoke(query)
    print(f'Время выполнения: {time.time() - current_time} секунд')
    print(search_type)
    print('=' * 10)




In [15]:
template = """
Определи содержатся ли в запросе слова, которые могут быть частью описания фильма

Если такие есть — перечисли их через запятую на английском. Если нет или сомневаешься — none
Отвечай в формате слова через запятую или none. Без окружающего текста и пояснений.
Например: Artist travels to the South или Best friends in the city.

Запрос: {query}
"""
prompt = PromptTemplate(template=template, input_variables=["query"])
llm_chain = prompt | llm
current_time = time.time()
queries = ["Найти фильмы про звездные войны",
           "Фильм snatch",
           "Найти улиточек с именем Наталии Деппа после 2020 года",
           "Что-нибудь про войну c высоким рейтингом"]

for query in queries:
    current_time = time.time()
    search_type = llm_chain.invoke(query)
    print(f'Время выполнения: {time.time() - current_time} секунд')
    print(search_type)
    print('=' * 10)




# YANDEX GPT 2

In [83]:
llm = YandexGPT(api_key=settings.yc_api_key,
                model_uri="gpt://b1gsfbseql2uigg6q996/yandexgpt-lite/latest",
                top_p=0.1,
                temperature=0.1,
                max_tokens=20)

In [44]:
template = """
Определи с поиском чего связан следущий запрос: поиск фильмов, поиск жанров или поиск персон.
Запрос: {query}
Если ты сомневаешься — отвечай none.
У тебя есть 4 варианта ответа: movies, genres, persons, none. Ответь, пожалуйста, одним словом.
"""
prompt = PromptTemplate(template=template, input_variables=["query"])
llm_chain = prompt | llm
current_time = time.time()
queries = ["Найти фильмы с участием актера Джонни Деппа",
           "Найти фильмы без титров",
           "Найти улиточек с именем Наталии Деппа",
           "Что-нибудь про войну"]

for query in queries:
    current_time = time.time()
    search_type = llm_chain.invoke(query)
    print(f'Время выполнения: {time.time() - current_time} секунд')
    print(search_type)
    print('=' * 10)




In [45]:
template = """
Определи содержится ли в данном запросе уточнение по свойству: имя человека
Запрос: {query}
Если ты сомневаешься — отвечай none.
Если есть — ответь через запятую строками для фильтра переведенными на английский язык.
Без лишних слов, только значения фильтра через запятую или none
"""
prompt = PromptTemplate(template=template, input_variables=["query"])
llm_chain = prompt | llm
current_time = time.time()
queries = ["Найти фильмы с участием актера Джонни Деппа",
           "Найти фильмы без титров",
           "Найти улиточек с именем Наталии Деппа",
           "Что-нибудь про войну"]

for query in queries:
    current_time = time.time()
    search_type = llm_chain.invoke(query)
    print(f'Время выполнения: {time.time() - current_time} секунд')
    print(search_type)
    print('=' * 10)




In [46]:
template = """
Определи содержится ли в данном запросе уточнение по упорядочиванию результатов по рейтингу
Запрос: {query}
Если ты сомневаешься — отвечай none.
Отвечай отднословно: asc, desc или none
"""
prompt = PromptTemplate(template=template, input_variables=["query"])
llm_chain = prompt | llm
current_time = time.time()
queries = ["Найти фильмы с участием актера Джонни Деппа",
           "Найти фильмы без титров",
           "Найти улиточек с именем Наталии Деппа",
           "Что-нибудь про войну c высоким рейтингом"]

for query in queries:
    current_time = time.time()
    search_type = llm_chain.invoke(query)
    print(f'Время выполнения: {time.time() - current_time} секунд')
    print(search_type)
    print('=' * 10)




In [47]:
template = """
Определи содержится ли в данном запросе уточнение по дате выхода фильма
Запрос: {query}
Если ты сомневаешься — отвечай none.
Отвечай в формате: gte:2020-01-01, lte:2021-01-01 или none
"""
prompt = PromptTemplate(template=template, input_variables=["query"])
llm_chain = prompt | llm
current_time = time.time()
queries = ["Найти фильмы с участием актера Джонни Деппа",
           "Найти фильмы без титров",
           "Найти улиточек с именем Наталии Деппа после 2020 года",
           "Что-нибудь про войну c высоким рейтингом"]

for query in queries:
    current_time = time.time()
    search_type = llm_chain.invoke(query)
    print(f'Время выполнения: {time.time() - current_time} секунд')
    print(search_type)
    print('=' * 10)




In [48]:
template = """
Определи содержится ли в данном запросе уточнение по жанру или жанрам фильма
Запрос: {query}
Если ты сомневаешься — отвечай none.
допустимые значения: action, adventure, animation, biography, comedy, crime, drama, family, fantasy, history, horror, music, musical, mystery, romance, sci-fi, sport, thriller, war, western
Отвечай в формате: значений через запятую или none
"""
prompt = PromptTemplate(template=template, input_variables=["query"])
llm_chain = prompt | llm
current_time = time.time()
queries = ["Найти фильмы с участием актера Джонни Деппа",
           "Найти фильмы без титров",
           "Найти улиточек с именем Наталии Деппа после 2020 года",
           "Что-нибудь про войну c высоким рейтингом"]

for query in queries:
    current_time = time.time()
    search_type = llm_chain.invoke(query)
    print(f'Время выполнения: {time.time() - current_time} секунд')
    print(search_type)
    print('=' * 10)




In [84]:
template = """
Определи содержатся ли в запросе слова, которые могут быть частью названия фильма

Если такие есть — перечисли их через запятую на английском. Если нет или сомневаешься — none
Отвечай в формате слова через запятую или none. Без окружающего текста и пояснений.

Запрос: {query}
"""
prompt = PromptTemplate(template=template, input_variables=["query"])
llm_chain = prompt | llm
current_time = time.time()
queries = ["Найти фильмы про звездные войны",
           "Фильм спиздили",
           "Найти мультфильмы про звездные войны",
           "Что-нибудь про войну c высоким рейтингом"]

for query in queries:
    current_time = time.time()
    search_type = llm_chain.invoke(query)
    print(f'Время выполнения: {time.time() - current_time} секунд')
    print(search_type)
    print('=' * 10)




In [85]:
template = """
Определи содержатся ли в запросе слова, которые могут быть частью описания фильма

Если такие есть — перечисли их через запятую на английском. Если нет или сомневаешься — none
Отвечай в формате слова через запятую или none. Без окружающего текста и пояснений.

Запрос: {query}
"""
prompt = PromptTemplate(template=template, input_variables=["query"])
llm_chain = prompt | llm
current_time = time.time()
queries = ["Найти фильмы про звездные войны",
           "Фильм спиздили",
           "Найти улиточек с именем Наталии Деппа после 2020 года",
           "Что-нибудь про войну c высоким рейтингом"]

for query in queries:
    current_time = time.time()
    search_type = llm_chain.invoke(query)
    print(f'Время выполнения: {time.time() - current_time} секунд')
    print(search_type)
    print('=' * 10)




# REKA

In [20]:
!pip install reka-api

In [21]:
import reka
reka.API_KEY = settings.reka_api_key

In [33]:
template_string = """
Определи содержатся ли в запросе слова, которые могут быть частью описания фильма

Если такие есть — перечисли их через запятую на английском. Если нет или сомневаешься — none
Отвечай в формате слова через запятую или none. Без окружающего текста и пояснений.
Например: Artist travels to the South или Best friends in the city.

Запрос: 
"""
queries = ["Найти фильмы про звездные войны",
           "Фильм snatch",
           "Найти улиточек с именем Наталии Деппа после 2020 года",
           "Что-нибудь про войну c высоким рейтингом"]

for query in queries:
    current_time = time.time()
    search_type = reka.chat(template_string + query, temperature=0.1, request_output_len=10)
    print(f'Время выполнения: {time.time() - current_time} секунд')
    print(search_type['text'].strip())
    print('=' * 10)


In [31]:
template_string = """
Определи с поиском чего связан следущий запрос: поиск фильмов, поиск жанров или поиск персон.

Если ты сомневаешься — отвечай none.
У тебя есть 4 варианта ответа: movies, genres, persons, none. Ответь, пожалуйста, одним словом.

Запрос: 
"""
queries = ["Найти фильмы про звездные войны",
           "Фильм snatch",
           "Найти улиточек с именем Наталии Деппа после 2020 года",
           "Что-нибудь про войну c высоким рейтингом"]

for query in queries:
    current_time = time.time()
    search_type = reka.chat(template_string + query, temperature=0.1, request_output_len=1)
    print(f'Время выполнения: {time.time() - current_time} секунд')
    print(search_type['text'].strip())
    print('=' * 10)




In [38]:
template_string = """
Определи содержится ли в данном запросе уточнение по свойству: имя человека

Если ты сомневаешься — отвечай none.
Если есть — ответь через запятую строками для фильтра переведенными на английский язык.
Без лишних слов, имена людей через запятую или none. Например Joe Lucas, Natalie Portman.
Запрос: 
"""
queries = ["Найти фильмы про звездные войны",
           "Фильм snatch",
           "Найти улиточек с именем Наталии Деппа после 2020 года",
           "Что-нибудь про войну c высоким рейтингом"]

for query in queries:
    current_time = time.time()
    search_type = reka.chat(template_string + query, temperature=0.1, request_output_len=10)
    print(f'Время выполнения: {time.time() - current_time} секунд')
    print(search_type['text'].strip())
    print('=' * 10)


